## 외부변수 추가 & 변수 별 가중치 적용하기

#### 목차
1) 데이터 불러오기
2) 누락값 처리
3) 외부변수 추가
4) 가중치 생성
5) 가중치 적용하고 상관성 확인


---

### 1) 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import datetime
plt.rc('font', family='NanumSquare')
import seaborn as sns
import statsmodels.api as sm

In [ ]:
items = pd.read_pickle('/content/gdrive/MyDrive/유통 데이터 공모전/pkl_data/예측상품_10개.pkl')
items

FileNotFoundError: ignored

In [ ]:
df = pd.read_pickle('/content/gdrive/MyDrive/유통 데이터 공모전/pkl_data/중소유통물류_전체상품.pkl')
df = df[df['구분'] == '매출']
df

,판매일,구분,우편번호,판매수량,옵션코드,규격,입수,상품 바코드,상품명
0,2021-01-04,매출,37542,4,EA,100g*1,1,8.801007e+12,다시다)쇠골드명품100g
1,2021-01-04,매출,37542,2,BX,12입,12,1.880104e+13,사조]해표식용유<1.8L>
2,2021-01-04,매출,37542,1,EA,1입,1,8.809199e+12,젓가락]키친센스 나무<250p>
3,2021-01-04,매출,37542,1,EA,50g*16*1,1,8.801077e+12,소세지 천하장사800g
4,2021-01-04,매출,37542,3,EA,10입,1,8.801045e+12,오뚜기]옛날자른당면<1kg >
...,...,...,...,...,...,...,...,...,...
521978,2022-07-01,매출,37924,3,CS,2L*6,6,8.808244e+12,삼다수2L
521979,2022-07-01,매출,37924,60,EA,30입,1,8.801106e+12,해태]업소용 매실<175ml>
521980,2022-07-01,매출,37924,1,BX,10*5CS,50,7.880106e+13,롯데]칸타타파우치 헤이즐넛<230ml>
521981,2022-07-01,매출,37924,6,EA,15T*1,1,8.802783e+12,오뚜기]차 대추차<15T>


In [ ]:
df.loc[df[df['옵션코드'] == 'BOX'].index.tolist(), '옵션코드'] = 'BX'
df.loc[df[df['옵션코드'] == 'C/S'].index.tolist(), '옵션코드'] = 'CS'

df = df[['판매일', '판매수량', '옵션코드', '입수', '상품 바코드', '상품명']]

df_new = pd.merge(df, items['상품 바코드'], how='inner', on='상품 바코드')

df_new = df_new.set_index(df_new['판매일'])
df_new.index = df_new.index.to_period(freq='W').to_timestamp(how='start')
df_new.index.name = 'date_w'
df_new.drop(['판매일', '입수'], axis=1, inplace=True)
train = df_new.reset_index()

train_gp = train.sort_values('date_w').groupby(['상품 바코드', '상품명','옵션코드','date_w'], as_index=False)
train_gp = train_gp.agg({'판매수량':['sum']})
train_gp.columns = ['상품 바코드','상품명','옵션코드', 'date_w', '판매수량']
train_gp

,상품 바코드,상품명,옵션코드,date_w,판매수량
0,8.801043e+12,농심]안성탕면 멀티<40>,BX,2021-01-04,48
1,8.801043e+12,농심]안성탕면 멀티<40>,BX,2021-01-11,51
2,8.801043e+12,농심]안성탕면 멀티<40>,BX,2021-01-18,71
3,8.801043e+12,농심]안성탕면 멀티<40>,BX,2021-01-25,54
4,8.801043e+12,농심]안성탕면 멀티<40>,BX,2021-02-01,72
...,...,...,...,...,...
773,6.880106e+13,레쓰비<175ml*30>,BX,2022-05-30,56
774,6.880106e+13,레쓰비<175ml*30>,BX,2022-06-06,77
775,6.880106e+13,레쓰비<175ml*30>,BX,2022-06-13,110
776,6.880106e+13,레쓰비<175ml*30>,BX,2022-06-20,120


### 2) 누락값 처리하기

In [ ]:
a = train_gp.groupby(['상품 바코드','상품명','옵션코드'])[['date_w']].count().reset_index()

In [ ]:
a

,상품 바코드,상품명,옵션코드,date_w
0,8.801043e+12,농심]안성탕면 멀티<40>,BX,78
1,8.801043e+12,농심]신라면 멀티<40>,BX,77
2,8.801043e+12,농심]너구리 얼큰멀티<40>,BX,78
3,8.801043e+12,농심]짜파게티 멀티<40>,BX,78
4,8.801094e+12,코카콜라<1.5*12>,EA,78
5,8.806012e+12,박카스F 120ml,CS,78
6,8.808244e+12,삼다수2L,CS,77
7,8.809274e+12,가야산천년수<2L*6>,CS,78
8,1.880105e+13,오뚜기]진라면매운멀티<120g>,BX,78
9,6.880106e+13,레쓰비<175ml*30>,BX,78


In [ ]:
base = pd.DataFrame()
for i in range(len(a)):
    n = pd.DataFrame()
    n['date_w'] = pd.date_range(start='2021-01-04', periods=78, freq='W-Mon').tolist()
    n['상품 바코드'] = a.loc[i, '상품 바코드']
    n['상품명'] = a.loc[i, '상품명']
    n['옵션코드'] = a.loc[i, '옵션코드']
    base = pd.concat([base,n])

train_gp = pd.merge(base, train_gp, how='left', on = ['상품 바코드', 'date_w', '옵션코드']).drop('상품명_y', axis=1)

In [ ]:
train_gp

,date_w,상품 바코드,상품명_x,옵션코드,판매수량
0,2021-01-04,8.801043e+12,농심]안성탕면 멀티<40>,BX,48.0
1,2021-01-11,8.801043e+12,농심]안성탕면 멀티<40>,BX,51.0
2,2021-01-18,8.801043e+12,농심]안성탕면 멀티<40>,BX,71.0
3,2021-01-25,8.801043e+12,농심]안성탕면 멀티<40>,BX,54.0
4,2021-02-01,8.801043e+12,농심]안성탕면 멀티<40>,BX,72.0
...,...,...,...,...,...
775,2022-05-30,6.880106e+13,레쓰비<175ml*30>,BX,56.0
776,2022-06-06,6.880106e+13,레쓰비<175ml*30>,BX,77.0
777,2022-06-13,6.880106e+13,레쓰비<175ml*30>,BX,110.0
778,2022-06-20,6.880106e+13,레쓰비<175ml*30>,BX,120.0


In [ ]:
# 농심]신라면 멀티<40>
train_gp.loc[98, '판매수량'] = 9

# 삼다수2L
train_gp.loc[524, '판매수량'] = 220

# 앞뒤 값의 평균을 줌

In [ ]:
b = train_gp.groupby(['상품 바코드','상품명_x','옵션코드'])[['date_w']].count().reset_index()
b

,상품 바코드,상품명_x,옵션코드,date_w
0,8.801043e+12,농심]안성탕면 멀티<40>,BX,78
1,8.801043e+12,농심]신라면 멀티<40>,BX,78
2,8.801043e+12,농심]너구리 얼큰멀티<40>,BX,78
3,8.801043e+12,농심]짜파게티 멀티<40>,BX,78
4,8.801094e+12,코카콜라<1.5*12>,EA,78
5,8.806012e+12,박카스F 120ml,CS,78
6,8.808244e+12,삼다수2L,CS,78
7,8.809274e+12,가야산천년수<2L*6>,CS,78
8,1.880105e+13,오뚜기]진라면매운멀티<120g>,BX,78
9,6.880106e+13,레쓰비<175ml*30>,BX,78


### 3) 외부 변수 추가 (지표까지 다 가져오기)

In [ ]:
holiday = pd.read_pickle('./pkl_data/공휴일정보.pkl')
holiday.index = holiday['날짜']
holiday.index = holiday.index.to_period(freq='W').to_timestamp(how='start')
holiday.drop('날짜', axis=1, inplace=True)
holiday = holiday.reset_index()
holiday = holiday.groupby('날짜').count().reset_index()

train_gp = pd.merge(train_gp, holiday, how='left', left_on='date_w', right_on='날짜')
train_gp['공휴일'] = train_gp['공휴일'].fillna(0)
train_gp.drop('날짜', axis=1, inplace=True)

train_gp['공휴일_여부'] = train_gp['공휴일'].apply(lambda x: 1 if x >= 1 else x)

In [ ]:
weather = pd.read_csv('./pkl_data/호미곶 날씨.csv', encoding='cp949')
weather['일시'] = pd.to_datetime(weather['일시'])
weather.index = weather['일시']
weather.index = weather.index.to_period(freq='W').to_timestamp(how='start')
weather.drop('일시', axis=1, inplace=True)
weather = weather.reset_index()
weather = weather.groupby('일시').mean().reset_index()

train_gp = pd.merge(train_gp, weather, how='left', left_on='date_w', right_on='일시')
train_gp.drop('일시', axis=1, inplace=True)

In [ ]:
train_gp.drop('지점', axis=1, inplace=True)

In [ ]:
covid = pd.read_pickle('./pkl_data/경북코로나확진자.pkl')
covid.index = covid['판매일']
covid.index = covid.index.to_period(freq='W').to_timestamp(how='start')
covid.drop('판매일', axis=1, inplace=True)
covid = covid.reset_index()
covid = covid.groupby('판매일').mean().reset_index()

train_gp = pd.merge(train_gp, covid, how='left', left_on='date_w', right_on='판매일')
train_gp.drop('판매일', axis=1, inplace=True)

In [ ]:
train_gp['년도'] = train_gp['date_w'].dt.year
train_gp['분기'] = train_gp['date_w'].dt.quarter
train_gp['월'] = train_gp['date_w'].dt.month
train_gp['주'] = train_gp['date_w'].dt.week

In [ ]:
economy = pd.read_excel('./pkl_data/한국경제지표(조정).xlsx')

economy['항목명3'] = economy['항목명3'].replace('/','-')
economy['항목명3'] = pd.to_datetime(economy['항목명3'], format= '%Y-%m')
economy['년도'] = economy['항목명3'].dt.year
economy['월'] = economy['항목명3'].dt.month
economy.drop(['항목명3'], axis=1, inplace=True)

train_gp = pd.merge(train_gp, economy, how='left', on=['년도','월'])
train_gp

,date_w,상품 바코드,상품명_x,옵션코드,판매수량,공휴일,공휴일_여부,평균기온(°C),최저기온(°C),최고기온(°C),...,분기,월,주,소비자심리지수,경제심리지수(원계열),경제심리지수(순환변동치),개인 대형마트/유통전문점 총액,개인 대형마트/유통전문점 월간 일평균,개인 신용카드 슈퍼마켓 총액,개인 신용카드 슈퍼마켓 월간 일평균
0,2021-01-04,8.801043e+12,농심]안성탕면 멀티<40>,BX,48.0,0.0,0.0,-2.942857,-6.671429,1.757143,...,1,1,1,95.3,92.6,93.5,89852,2898,78905,2545
1,2021-01-11,8.801043e+12,농심]안성탕면 멀티<40>,BX,51.0,0.0,0.0,4.200000,-0.742857,9.171429,...,1,1,2,95.3,92.6,93.5,89852,2898,78905,2545
2,2021-01-18,8.801043e+12,농심]안성탕면 멀티<40>,BX,71.0,0.0,0.0,6.528571,3.014286,10.171429,...,1,1,3,95.3,92.6,93.5,89852,2898,78905,2545
3,2021-01-25,8.801043e+12,농심]안성탕면 멀티<40>,BX,54.0,0.0,0.0,5.514286,1.500000,9.571429,...,1,1,4,95.3,92.6,93.5,89852,2898,78905,2545
4,2021-02-01,8.801043e+12,농심]안성탕면 멀티<40>,BX,72.0,0.0,0.0,5.742857,1.685714,10.942857,...,1,2,5,97.4,96.2,96.4,89222,3186,71806,2564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2022-05-30,6.880106e+13,레쓰비<175ml*30>,BX,56.0,1.0,1.0,20.228571,16.585714,24.700000,...,2,5,22,102.9,106.8,102.4,102491,3306,87008,2807
776,2022-06-06,6.880106e+13,레쓰비<175ml*30>,BX,77.0,1.0,1.0,18.585714,15.914286,22.014286,...,2,6,23,96.7,102.6,101.1,95686,3190,83747,2792
777,2022-06-13,6.880106e+13,레쓰비<175ml*30>,BX,110.0,0.0,0.0,22.300000,18.300000,26.357143,...,2,6,24,96.7,102.6,101.1,95686,3190,83747,2792
778,2022-06-20,6.880106e+13,레쓰비<175ml*30>,BX,120.0,0.0,0.0,25.800000,20.671429,31.328571,...,2,6,25,96.7,102.6,101.1,95686,3190,83747,2792


In [ ]:
info = pd.read_excel('./pkl_data/경제지수_변수.xlsx',index_col=0)
info['년도'] = info.index.year
info['월'] = info.index.month
train_gp = pd.merge(train_gp, info, how='left', on=['년도', '월'])

In [ ]:
stock_gaya = pd.read_excel('./pkl_data/가야산천연수_주가.xlsx')
stock_gaya['일자'] = pd.to_datetime(stock_gaya['일자'])
stock_gaya.index = stock_gaya['일자']
stock_gaya.index = stock_gaya.index.to_period(freq='W').to_timestamp(how='start')
stock_gaya.drop('일자', axis=1, inplace=True)
stock_gaya = stock_gaya.reset_index()

In [ ]:
stock_gaya_info = pd.DataFrame()
stock_gaya_info.index = stock_gaya.groupby('일자').min()['종가'].index
stock_gaya_info['stock_gaya_min'] = stock_gaya.groupby('일자').min()['종가']
stock_gaya_info['stock_gaya_max'] = stock_gaya.groupby('일자').max()['종가']
stock_gaya_info['stock_gaya_mean'] = stock_gaya.groupby('일자').mean()['종가']
stock_gaya_info = stock_gaya_info.reset_index()

In [ ]:
train_gp = pd.merge(train_gp, stock_gaya_info, how='left', left_on='date_w', right_on='일자')

In [ ]:
# 코카콜라
stock_coke = pd.read_excel('./pkl_data/코카콜라_주가.xlsx')
stock_coke = stock_coke[['날짜', '종가']]
stock_coke.index = stock_coke['날짜']
stock_coke.index = stock_coke.index.to_period(freq='W').to_timestamp(how='start')
stock_coke.drop('날짜', axis=1, inplace=True)
stock_coke = stock_coke.reset_index()

stock_coke_info = pd.DataFrame()
stock_coke_info.index = stock_coke.groupby('날짜').min()['종가'].index
stock_coke_info['stock_coke_min'] = stock_coke.groupby('날짜').min()['종가']
stock_coke_info['stock_coke_max'] = stock_coke.groupby('날짜').max()['종가']
stock_coke_info['stock_coke_mean'] = stock_coke.groupby('날짜').mean()['종가']
stock_coke_info = stock_coke_info.reset_index()

In [ ]:
train_gp = pd.merge(train_gp, stock_coke_info, how='left', left_on='date_w', right_on='날짜')

In [ ]:
# 삼다수
stock_samda = pd.read_excel('./pkl_data/samdasu_stock.xlsx')
stock_samda.index = stock_samda['일자']
stock_samda.index = stock_samda.index.to_period(freq='W').to_timestamp(how='start')
stock_samda.drop('일자', axis=1, inplace=True)
stock_samda = stock_samda.reset_index()

In [ ]:
stock_samda_info = pd.DataFrame()
stock_samda_info.index = stock_samda.groupby('일자').min()['종가'].index
stock_samda_info['stock_samda_min'] = stock_samda.groupby('일자').min()['종가']
stock_samda_info['stock_samda_max'] = stock_samda.groupby('일자').max()['종가']
stock_samda_info['stock_samda_mean'] = stock_samda.groupby('일자').mean()['종가']
stock_samda_info = stock_samda_info.reset_index()

In [ ]:
train_gp = pd.merge(train_gp, stock_samda_info, how='left', left_on='date_w', right_on='일자')

In [ ]:
train_new = train_gp.copy()
train_new

,date_w,상품 바코드,상품명_x,옵션코드,판매수량,공휴일,공휴일_여부,평균기온(°C),최저기온(°C),최고기온(°C),...,stock_gaya_max,stock_gaya_mean,날짜,stock_coke_min,stock_coke_max,stock_coke_mean,일자_y,stock_samda_min,stock_samda_max,stock_samda_mean
0,2021-01-04,8.801043e+12,농심]안성탕면 멀티<40>,BX,48.0,0.0,0.0,-2.942857,-6.671429,1.757143,...,124000,122600.0,2021-01-04,49.96,52.76,51.3000,2021-01-04,10100,10600,10360.0
1,2021-01-11,8.801043e+12,농심]안성탕면 멀티<40>,BX,51.0,0.0,0.0,4.200000,-0.742857,9.171429,...,130500,128900.0,2021-01-11,48.70,50.20,49.6800,2021-01-11,9820,10000,9922.0
2,2021-01-18,8.801043e+12,농심]안성탕면 멀티<40>,BX,71.0,0.0,0.0,6.528571,3.014286,10.171429,...,128500,124900.0,2021-01-18,48.49,48.95,48.6575,2021-01-18,9430,9640,9552.0
3,2021-01-25,8.801043e+12,농심]안성탕면 멀티<40>,BX,54.0,0.0,0.0,5.514286,1.500000,9.571429,...,126000,121600.0,2021-01-25,48.15,49.29,48.7800,2021-01-25,9300,9630,9464.0
4,2021-02-01,8.801043e+12,농심]안성탕면 멀티<40>,BX,72.0,0.0,0.0,5.742857,1.685714,10.942857,...,114000,111600.0,2021-02-01,48.48,49.65,48.9740,2021-02-01,9300,9400,9350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2022-05-30,6.880106e+13,레쓰비<175ml*30>,BX,56.0,1.0,1.0,20.228571,16.585714,24.700000,...,107000,106250.0,2022-05-30,62.97,63.73,63.2875,2022-05-30,6950,7000,6972.5
776,2022-06-06,6.880106e+13,레쓰비<175ml*30>,BX,77.0,1.0,1.0,18.585714,15.914286,22.014286,...,106000,104750.0,2022-06-06,61.41,63.25,62.4440,2022-06-06,6730,6920,6822.5
777,2022-06-13,6.880106e+13,레쓰비<175ml*30>,BX,110.0,0.0,0.0,22.300000,18.300000,26.357143,...,103000,100780.0,2022-06-13,59.07,61.34,59.7480,2022-06-13,6060,6540,6250.0
778,2022-06-20,6.880106e+13,레쓰비<175ml*30>,BX,120.0,0.0,0.0,25.800000,20.671429,31.328571,...,104000,101280.0,2022-06-20,60.70,63.04,61.6925,2022-06-20,5820,6160,6018.0


In [ ]:
train_new.drop(['일자_x','날짜' ,'일자_y'], axis=1, inplace=True)
train_new = train_new.rename(columns={'상품명_x':'상품명'})

In [ ]:
train = train_new.copy()

In [ ]:
# 참고데이터 변수까지
df = pd.read_pickle('./pkl_data/지역마트판매.pkl')

df = df[(df['CATEGORY'] == '음료') | (df['CATEGORY'] == '라면')]
df = df.reset_index(drop=True)

df['STARTDATE'] = pd.to_datetime(df['STARTDATE'], format = '%Y%m%d')
df['ENDDATE'] = pd.to_datetime(df['ENDDATE'], format = '%Y%m%d')
df['YEAR'] = df['MONTHLY'].str[1:5].astype('int64')
df['MONTH'] = df['MONTHLY'].str[5:].astype('int64')
df['WEEK'] = df['WEEKLY'].str[5:].astype('int64')
df['QUARTER'] = df['STARTDATE'].dt.quarter
df['PRICE'] = df['AMOUNT'] / df['QUANTITY']

In [ ]:
def name(item):
    if item == '레쓰비 마일드커피 175ML*6':
        name = '레쓰비<175ml*30>'
    elif item == '박카스F 120mL':
        name = '박카스F 120ml'
    elif item == '백산수 330mL':
        name = '가야산천년수<2L*6>'
    elif item == '신라면 120G*5':
        name = '농심]신라면 멀티<40>'
    elif item == '안성탕면 125G*5':
        name = '농심]안성탕면 멀티<40>'
    elif item == '얼큰한 너구리 120G*5':
        name = '농심]너구리 얼큰멀티<40>'
    elif item == '올리브 짜파게티 140G*5':
        name = '농심]짜파게티 멀티<40>'
    elif item == '제주삼다수 500mL':
        name = '삼다수2L'
    elif item == '진라면 매운맛 120G*5':
        name = '오뚜기]진라면매운멀티<120g>'
    elif item == '코카콜라 215mL*6':
        name = '코카콜라<1.5*12>'
    return name

In [ ]:
#상품별 가중치 만들기 - 전체상품의 분기/월/주 별 매출정보(mean)에다가 상품별 가중치 적용
weight_df = df[df['ITEM_DESCRIPTION'].duplicated() == False][['ITEM_DESCRIPTION']].reset_index(drop=True)
weight_df['상품명'] = weight_df['ITEM_DESCRIPTION'].apply(name)

df1 = df.groupby(['ITEM_DESCRIPTION', 'QUARTER'])[['QUANTITY', 'AMOUNT', 'PRICE']].sum().reset_index()
df2 = df.groupby(['ITEM_DESCRIPTION', 'MONTH'])[['QUANTITY', 'AMOUNT', 'PRICE']].sum().reset_index()
df3 = df.groupby(['ITEM_DESCRIPTION', 'WEEK'])[['QUANTITY', 'AMOUNT', 'PRICE']].sum().reset_index()

In [ ]:
df1['상품명'] = df1['ITEM_DESCRIPTION'].apply(name)
df2['상품명'] = df2['ITEM_DESCRIPTION'].apply(name)
df3['상품명'] = df3['ITEM_DESCRIPTION'].apply(name)

In [ ]:
df1 = df1.rename(columns={'QUANTITY':'분기별_QUANTITY', 'AMOUNT':'분기별_AMOUNT', 'PRICE':'분기별_PRICE'})
df2 = df2.rename(columns={'QUANTITY':'월별_QUANTITY', 'AMOUNT':'월별_AMOUNT', 'PRICE':'월별_PRICE'})
df3 = df3.rename(columns={'QUANTITY':'주별_QUANTITY', 'AMOUNT':'주별_AMOUNT', 'PRICE':'주별_PRICE'})

In [ ]:
train['WEEK'] = train['date_w'].dt.week
train['QUARTER'] = train['date_w'].dt.quarter
train['MONTH'] = train['date_w'].dt.month

train = pd.merge(train, df1.iloc[:,1:], how='left', on=['상품명', 'QUARTER'])
train = pd.merge(train, df2.iloc[:,1:], how='left', on=['상품명', 'MONTH'])
train = pd.merge(train, df3.iloc[:,1:], how='left', on=['상품명', 'WEEK'])

In [ ]:
train.drop(['최저기온(°C)', '평균기온(°C)'], axis=1, inplace=True)

In [ ]:
train.columns

Index(['date_w', '상품 바코드', '상품명', '옵션코드', '판매수량', '공휴일', '공휴일_여부', '최고기온(°C)',
       'patient', '년도', '분기', '월', '주', '소비자심리지수', '경제심리지수(원계열)',
       '경제심리지수(순환변동치)', '개인 대형마트/유통전문점 총액', '개인 대형마트/유통전문점 월간 일평균',
       '개인 신용카드 슈퍼마켓 총액', '개인 신용카드 슈퍼마켓 월간 일평균', '경기 종합', '경북경제활동', '경북고용률',
       '광공업생산지수', '서비스업생산지수', '제조업가동률지수', '소비자물가지수', '생산자물가지수', '광공업-생산확산지수',
       'stock_gaya_min', 'stock_gaya_max', 'stock_gaya_mean', 'stock_coke_min',
       'stock_coke_max', 'stock_coke_mean', 'stock_samda_min',
       'stock_samda_max', 'stock_samda_mean', 'WEEK', 'QUARTER', 'MONTH',
       '분기별_QUANTITY', '분기별_AMOUNT', '분기별_PRICE', '월별_QUANTITY', '월별_AMOUNT',
       '월별_PRICE', '주별_QUANTITY', '주별_AMOUNT', '주별_PRICE'],
      dtype='object')

### 4) 가중치 생성하기

In [ ]:
weight_df = train_new[train_new[['상품 바코드', '상품명']].duplicated() == False][['상품 바코드', '상품명']].reset_index(drop=True)

In [ ]:
holi = []
is_holi = []
temp = []
covid = []
year = []
quater = []
month = []
week = []

경기_종합 = []
경북경제활동 = []
경북고용률 = []
광공업생산지수 = []
서비스업생산지수 = []
제조업가동률지수 = []
소비자물가지수 = []
생산자물가지수 = []
광공업_생산확산지수 = []

소비자심리지수 = []
경제심리지수_원계열 = []
경제심리지수_순환변동치 = []
개인_대형마트_유통전문점_총액 = []
개인_대형마트_유통전문점_월간_일평균 = []
개인_신용카드_슈퍼마켓_총액 = []
개인_신용카드_슈퍼마켓_월간_일평균 = []

가야_min = []
가야_mean = []
가야_max = []

코카_min = []
코카_mean = []
코카_max = []

삼다수_min = []
삼다수_mean = []
삼다수_max = []

quarter_quantity = []
quarter_amount = []
quarter_price = []

month_quantity = []
month_amount = []
month_price = []

week_quantity = []
week_amount = []
week_price = []

In [ ]:
for i in range(len(weight_df)):
    item_df = train[train['상품 바코드'] == weight_df.loc[i, '상품 바코드']]
    item_corr = item_df.corr()['판매수량']
    holi.append(item_corr['공휴일'])
    is_holi.append(item_corr['공휴일_여부'])
    temp.append(item_corr['최고기온(°C)'])
    covid.append(item_corr['patient'])
    year.append(item_corr['년도'])
    quater.append(item_corr['분기'])
    month.append(item_corr['월'])
    week.append(item_corr['주'])

    경기_종합.append(item_corr['경기 종합'])
    경북경제활동.append(item_corr['경북경제활동'])
    경북고용률.append(item_corr['경북고용률'])
    광공업생산지수.append(item_corr['광공업생산지수'])
    서비스업생산지수.append(item_corr['서비스업생산지수'])
    제조업가동률지수.append(item_corr['제조업가동률지수'])
    소비자물가지수.append(item_corr['소비자물가지수'])
    생산자물가지수.append(item_corr['생산자물가지수'])
    광공업_생산확산지수.append(item_corr['광공업-생산확산지수'])

    소비자심리지수.append(item_corr['소비자심리지수'])
    경제심리지수_원계열.append(item_corr['경제심리지수(원계열)'])
    경제심리지수_순환변동치.append(item_corr['경제심리지수(순환변동치)'])
    개인_대형마트_유통전문점_총액.append(item_corr['개인 대형마트/유통전문점 총액'])
    개인_대형마트_유통전문점_월간_일평균.append(item_corr['개인 대형마트/유통전문점 월간 일평균'])
    개인_신용카드_슈퍼마켓_총액.append(item_corr['개인 신용카드 슈퍼마켓 총액'])
    개인_신용카드_슈퍼마켓_월간_일평균.append(item_corr['개인 신용카드 슈퍼마켓 월간 일평균'])

    가야_min.append(item_corr['stock_gaya_min'])
    가야_mean.append(item_corr['stock_gaya_mean'])
    가야_max.append(item_corr['stock_gaya_max'])

    코카_min.append(item_corr['stock_coke_min'])
    코카_mean.append(item_corr['stock_coke_mean'])
    코카_max.append(item_corr['stock_coke_max'])

    삼다수_min.append(item_corr['stock_samda_min'])
    삼다수_mean.append(item_corr['stock_samda_mean'])
    삼다수_max.append(item_corr['stock_samda_max'])

    quarter_quantity.append(item_corr['분기별_QUANTITY'])
    month_quantity.append(item_corr['월별_QUANTITY'])
    week_quantity.append(item_corr['주별_QUANTITY'])

    quarter_amount.append(item_corr['분기별_AMOUNT'])
    month_amount.append(item_corr['월별_AMOUNT'])
    week_amount.append(item_corr['주별_AMOUNT'])

    quarter_price.append(item_corr['분기별_PRICE'])
    month_price.append(item_corr['월별_PRICE'])
    week_price.append(item_corr['주별_PRICE'])

In [ ]:
weight_df['공휴일_가중치'] = holi
weight_df['공휴일_여부_가중치'] = is_holi
weight_df['최고기온_가중치']= temp
weight_df['patient_가중치']= covid

weight_df['년도_가중치'] = year
weight_df['분기_가중치'] = quater
weight_df['월_가중치']= month
weight_df['주_가중치']= week

weight_df['경기 종합_가중치'] = 경기_종합
weight_df['경북경제활동_가중치'] = 경북경제활동
weight_df['경북고용률_가중치']= 경북고용률
weight_df['광공업생산지수_가중치']= 광공업생산지수
weight_df['서비스업생산지수_가중치']= 서비스업생산지수
weight_df['제조업가동률지수_가중치']= 제조업가동률지수

weight_df['소비자물가지수_가중치'] = 소비자물가지수
weight_df['생산자물가지수_가중치'] = 생산자물가지수
weight_df['광공업-생산확산지수_가중치'] = 광공업_생산확산지수

weight_df['소비자심리지수_가중치'] = 소비자심리지수
weight_df['경제심리지수(원계열)_가중치'] = 경제심리지수_원계열
weight_df['경제심리지수(순환변동치)_가중치'] = 경제심리지수_순환변동치
weight_df['개인 대형마트/유통전문점 총액_가중치'] = 개인_대형마트_유통전문점_총액
weight_df['개인 대형마트/유통전문점 월간 일평균_가중치'] = 개인_대형마트_유통전문점_월간_일평균
weight_df['개인 신용카드 슈퍼마켓 총액_가중치'] = 개인_신용카드_슈퍼마켓_총액
weight_df['개인 신용카드 슈퍼마켓 월간 일평균_가중치'] = 개인_신용카드_슈퍼마켓_월간_일평균


weight_df['stock_gaya_min_가중치'] = 가야_min
weight_df['stock_gaya_max_가중치'] = 가야_max
weight_df['stock_gaya_mean_가중치'] = 가야_mean

weight_df['stock_samda_min_가중치'] = 삼다수_min
weight_df['stock_samda_max_가중치'] = 삼다수_max
weight_df['stock_samda_mean_가중치'] = 삼다수_mean

weight_df['stock_coke_min_가중치'] = 코카_min
weight_df['stock_coke_max_가중치'] = 코카_max
weight_df['stock_coke_mean_가중치'] = 코카_mean

weight_df['분기별_QUANTITY_가중치'] = quarter_quantity
weight_df['분기별_AMOUNT_가중치'] = quarter_amount
weight_df['분기별_PRICE_가중치'] = quarter_price

weight_df['월별_QUANTITY_가중치'] = month_quantity
weight_df['월별_AMOUNT_가중치'] = month_amount
weight_df['월별_PRICE_가중치'] = month_price

weight_df['주별_QUANTITY_가중치'] = week_quantity
weight_df['주별_AMOUNT_가중치'] = week_amount
weight_df['주별_PRICE_가중치'] = week_price

In [ ]:
weight_df

,상품 바코드,상품명,공휴일_가중치,공휴일_여부_가중치,최고기온_가중치,patient_가중치,년도_가중치,분기_가중치,월_가중치,주_가중치,...,stock_coke_mean_가중치,분기별_QUANTITY_가중치,분기별_AMOUNT_가중치,분기별_PRICE_가중치,월별_QUANTITY_가중치,월별_AMOUNT_가중치,월별_PRICE_가중치,주별_QUANTITY_가중치,주별_AMOUNT_가중치,주별_PRICE_가중치
0,8.801043e+12,농심]안성탕면 멀티<40>,0.018935,0.028361,0.154700,-0.058148,0.014938,0.086273,0.068759,0.070139,...,0.094216,0.109315,0.103091,0.102993,0.086283,0.080393,0.151033,0.120036,0.117775,0.216686
1,8.801043e+12,농심]신라면 멀티<40>,0.045027,0.074867,-0.007921,0.020643,0.023976,0.325617,0.328592,0.342958,...,0.160113,-0.272562,-0.283015,-0.294962,-0.211075,-0.229434,-0.318686,-0.085951,-0.101841,-0.342015
2,8.801043e+12,농심]너구리 얼큰멀티<40>,-0.122665,-0.064674,0.021887,-0.012871,-0.064901,0.046221,0.001135,-0.004910,...,-0.048674,-0.077692,-0.105839,-0.173044,-0.212364,-0.247814,-0.258612,-0.099175,-0.136030,-0.175740
3,8.801043e+12,농심]짜파게티 멀티<40>,-0.115347,-0.130636,0.066745,-0.109187,-0.132975,0.224395,0.203391,0.190037,...,-0.036848,-0.099942,-0.121869,-0.178417,-0.024244,-0.073871,-0.183063,-0.027350,-0.059476,-0.138075
4,8.801094e+12,코카콜라<1.5*12>,0.028559,0.096454,-0.378959,0.267197,0.681764,-0.186779,-0.183064,-0.176945,...,0.637831,-0.294546,-0.286593,-0.085127,-0.145802,-0.148614,-0.065439,-0.096877,-0.093747,-0.036835
5,8.806012e+12,박카스F 120ml,-0.245880,-0.142060,0.359850,-0.064284,-0.073832,0.241339,0.227305,0.224351,...,0.083877,0.346860,0.346930,0.310975,0.164190,0.163951,0.163841,0.077876,0.078013,0.150044
6,8.808244e+12,삼다수2L,0.055591,0.020901,-0.164811,0.249850,0.260431,-0.128490,-0.163097,-0.172504,...,0.209127,-0.117363,-0.115447,-0.171074,-0.284609,-0.285384,-0.362640,-0.178767,-0.177930,-0.311112
7,8.809274e+12,가야산천년수<2L*6>,-0.217515,-0.191543,0.551748,-0.238939,-0.244021,0.155245,0.115527,0.114984,...,-0.105097,0.408519,0.414490,0.298190,0.226246,0.229228,0.194447,0.236528,0.246690,0.242640
8,1.880105e+13,오뚜기]진라면매운멀티<120g>,-0.165612,-0.106634,0.196275,-0.192352,-0.085025,0.047889,0.020739,0.026229,...,0.005120,0.191364,0.177976,0.114559,0.061063,0.055029,0.120609,0.106153,0.101969,0.184308
9,6.880106e+13,레쓰비<175ml*30>,-0.118364,-0.063267,0.419958,-0.121108,-0.281726,0.293830,0.332784,0.320356,...,-0.155897,0.208632,0.227291,0.054008,0.252580,0.307242,0.188739,0.281540,0.348093,0.158129


### 5) 가중치 적용하고 상관성 확인

In [ ]:
train.columns

Index(['date_w', '상품 바코드', '상품명', '옵션코드', '판매수량', '공휴일', '공휴일_여부', '최고기온(°C)',
       'patient', '년도', '분기', '월', '주', '소비자심리지수', '경제심리지수(원계열)',
       '경제심리지수(순환변동치)', '개인 대형마트/유통전문점 총액', '개인 대형마트/유통전문점 월간 일평균',
       '개인 신용카드 슈퍼마켓 총액', '개인 신용카드 슈퍼마켓 월간 일평균', '경기 종합', '경북경제활동', '경북고용률',
       '광공업생산지수', '서비스업생산지수', '제조업가동률지수', '소비자물가지수', '생산자물가지수', '광공업-생산확산지수',
       'stock_gaya_min', 'stock_gaya_max', 'stock_gaya_mean', 'stock_coke_min',
       'stock_coke_max', 'stock_coke_mean', 'stock_samda_min',
       'stock_samda_max', 'stock_samda_mean', 'WEEK', 'QUARTER', 'MONTH',
       '분기별_QUANTITY', '분기별_AMOUNT', '분기별_PRICE', '월별_QUANTITY', '월별_AMOUNT',
       '월별_PRICE', '주별_QUANTITY', '주별_AMOUNT', '주별_PRICE'],
      dtype='object')

In [ ]:
train_new = pd.merge(train, weight_df, how='left', on=['상품 바코드', '상품명'])

In [ ]:
train_new['공휴일'] = train_new['공휴일'] * train_new['공휴일_가중치']
train_new['최고기온(°C)'] = train_new['최고기온(°C)'] * train_new['최고기온_가중치']
train_new['공휴일_여부'] = train_new['공휴일_여부'] * train_new['공휴일_여부_가중치']
train_new['patient'] = train_new['patient'] * train_new['patient_가중치']

train_new['년도'] = train_new['년도'] * train_new['년도_가중치']
train_new['분기'] = train_new['분기'] * train_new['분기_가중치']
train_new['월'] = train_new['월'] * train_new['월_가중치']
train_new['주'] = train_new['주'] * train_new['주_가중치']

train_new['경기 종합'] = train_new['경기 종합'] * train_new['경기 종합_가중치']
train_new['경북경제활동'] = train_new['경북경제활동'] * train_new['경북경제활동_가중치']
train_new['경북고용률'] = train_new['경북고용률'] * train_new['경북고용률_가중치']
train_new['광공업생산지수'] = train_new['광공업생산지수'] * train_new['광공업생산지수_가중치']
train_new['서비스업생산지수'] = train_new['서비스업생산지수'] * train_new['서비스업생산지수_가중치']
train_new['제조업가동률지수'] = train_new['제조업가동률지수'] * train_new['제조업가동률지수_가중치']

train_new['소비자물가지수'] = train_new['소비자물가지수'] * train_new['소비자물가지수_가중치']
train_new['생산자물가지수'] = train_new['생산자물가지수'] * train_new['생산자물가지수_가중치']
train_new['광공업-생산확산지수'] = train_new['광공업-생산확산지수'] * train_new['광공업-생산확산지수_가중치']

train_new['소비자심리지수'] = train_new['소비자심리지수'] * train_new['소비자심리지수_가중치']
train_new['경제심리지수(원계열)'] = train_new['경제심리지수(원계열)'] * train_new['경제심리지수(원계열)_가중치']
train_new['경제심리지수(순환변동치)'] = train_new['경제심리지수(순환변동치)'] * train_new['경제심리지수(순환변동치)_가중치']
train_new['개인 대형마트/유통전문점 총액'] = train_new['개인 대형마트/유통전문점 총액'] * train_new['개인 대형마트/유통전문점 총액_가중치']
train_new['개인 대형마트/유통전문점 월간 일평균'] = train_new['개인 대형마트/유통전문점 월간 일평균'] * train_new['개인 대형마트/유통전문점 월간 일평균_가중치']
train_new['개인 신용카드 슈퍼마켓 총액'] = train_new['개인 신용카드 슈퍼마켓 총액'] * train_new['개인 신용카드 슈퍼마켓 총액_가중치']
train_new['개인 신용카드 슈퍼마켓 월간 일평균'] = train_new['개인 신용카드 슈퍼마켓 월간 일평균'] * train_new['개인 신용카드 슈퍼마켓 월간 일평균_가중치']

train_new['stock_gaya_min'] = train_new['stock_gaya_min'] * train_new['stock_gaya_min_가중치']
train_new['stock_gaya_max'] = train_new['stock_gaya_max'] * train_new['stock_gaya_max_가중치']
train_new['stock_gaya_mean'] = train_new['stock_gaya_mean'] * train_new['stock_gaya_mean_가중치']

train_new['stock_samda_min'] = train_new['stock_samda_min'] * train_new['stock_samda_min_가중치']
train_new['stock_samda_max'] = train_new['stock_samda_max'] * train_new['stock_samda_max_가중치']
train_new['stock_samda_mean'] = train_new['stock_samda_mean'] * train_new['stock_samda_mean_가중치']

train_new['stock_coke_min'] = train_new['stock_coke_min'] * train_new['stock_coke_min_가중치']
train_new['stock_coke_max'] = train_new['stock_coke_max'] * train_new['stock_coke_max_가중치']
train_new['stock_coke_mean'] = train_new['stock_coke_mean'] * train_new['stock_coke_mean_가중치']

train_new['분기별_QUANTITY'] = train_new['분기별_QUANTITY'] * train_new['분기별_QUANTITY_가중치']
train_new['월별_QUANTITY'] = train_new['월별_QUANTITY'] * train_new['월별_QUANTITY_가중치']
train_new['주별_QUANTITY'] = train_new['주별_QUANTITY'] * train_new['주별_QUANTITY_가중치']

train_new['분기별_AMOUNT'] = train_new['분기별_AMOUNT'] * train_new['분기별_AMOUNT_가중치']
train_new['월별_AMOUNT'] = train_new['월별_AMOUNT'] * train_new['월별_AMOUNT_가중치']
train_new['주별_AMOUNT'] = train_new['주별_AMOUNT'] * train_new['주별_AMOUNT_가중치']

train_new['분기별_PRICE'] = train_new['분기별_PRICE'] * train_new['분기별_PRICE_가중치']
train_new['월별_PRICE'] = train_new['월별_PRICE'] * train_new['월별_PRICE_가중치']
train_new['주별_PRICE'] = train_new['주별_PRICE'] * train_new['주별_PRICE_가중치']

In [ ]:
train_new.columns

Index(['date_w', '상품 바코드', '상품명', '옵션코드', '판매수량', '공휴일', '공휴일_여부', '최고기온(°C)',
       'patient', '년도', '분기', '월', '주', '소비자심리지수', '경제심리지수(원계열)',
       '경제심리지수(순환변동치)', '개인 대형마트/유통전문점 총액', '개인 대형마트/유통전문점 월간 일평균',
       '개인 신용카드 슈퍼마켓 총액', '개인 신용카드 슈퍼마켓 월간 일평균', '경기 종합', '경북경제활동', '경북고용률',
       '광공업생산지수', '서비스업생산지수', '제조업가동률지수', '소비자물가지수', '생산자물가지수', '광공업-생산확산지수',
       'stock_gaya_min', 'stock_gaya_max', 'stock_gaya_mean', 'stock_coke_min',
       'stock_coke_max', 'stock_coke_mean', 'stock_samda_min',
       'stock_samda_max', 'stock_samda_mean', 'WEEK', 'QUARTER', 'MONTH',
       '분기별_QUANTITY', '분기별_AMOUNT', '분기별_PRICE', '월별_QUANTITY', '월별_AMOUNT',
       '월별_PRICE', '주별_QUANTITY', '주별_AMOUNT', '주별_PRICE', '공휴일_가중치',
       '공휴일_여부_가중치', '최고기온_가중치', 'patient_가중치', '년도_가중치', '분기_가중치', '월_가중치',
       '주_가중치', '경기 종합_가중치', '경북경제활동_가중치', '경북고용률_가중치', '광공업생산지수_가중치',
       '서비스업생산지수_가중치', '제조업가동률지수_가중치', '소비자물가지수_가중치', '생산자물가지수_가중치',
       '광공업-생산확산지수_가중치', '소비자심리지수_가중치', '경제심리지수(원계

In [ ]:
train_new.drop(['공휴일_가중치',
       '공휴일_여부_가중치', '최고기온_가중치', 'patient_가중치', '년도_가중치', '분기_가중치', '월_가중치',
       '주_가중치', '경기 종합_가중치', '경북경제활동_가중치', '경북고용률_가중치', '광공업생산지수_가중치',
       '서비스업생산지수_가중치', '제조업가동률지수_가중치', '소비자물가지수_가중치', '생산자물가지수_가중치',
       '광공업-생산확산지수_가중치', '소비자심리지수_가중치', '경제심리지수(원계열)_가중치', '경제심리지수(순환변동치)_가중치',
       '개인 대형마트/유통전문점 총액_가중치', '개인 대형마트/유통전문점 월간 일평균_가중치',
       '개인 신용카드 슈퍼마켓 총액_가중치', '개인 신용카드 슈퍼마켓 월간 일평균_가중치', 'stock_gaya_min_가중치',
       'stock_gaya_max_가중치', 'stock_gaya_mean_가중치', 'stock_samda_min_가중치',
       'stock_samda_max_가중치', 'stock_samda_mean_가중치', 'stock_coke_min_가중치',
       'stock_coke_max_가중치', 'stock_coke_mean_가중치', '분기별_QUANTITY_가중치',
       '분기별_AMOUNT_가중치', '분기별_PRICE_가중치', '월별_QUANTITY_가중치', '월별_AMOUNT_가중치',
       '월별_PRICE_가중치', '주별_QUANTITY_가중치', '주별_AMOUNT_가중치', '주별_PRICE_가중치'],axis=1, inplace=True)

In [ ]:
train_new.corr()

,상품 바코드,판매수량,공휴일,공휴일_여부,최고기온(°C),patient,년도,분기,월,주,...,MONTH,분기별_QUANTITY,분기별_AMOUNT,분기별_PRICE,월별_QUANTITY,월별_AMOUNT,월별_PRICE,주별_QUANTITY,주별_AMOUNT,주별_PRICE
상품 바코드,1.000000e+00,-0.147118,-0.054106,-0.040936,0.358091,-0.099047,-0.387488,0.304908,0.359355,0.336129,...,2.153145e-16,0.120871,0.196136,0.291080,0.182772,0.236904,0.432534,0.128541,0.191148,0.355066
판매수량,-1.471176e-01,1.000000,-0.043601,-0.105724,0.364914,-0.005352,-0.073210,-0.075384,-0.098426,-0.092834,...,9.478960e-03,0.285723,0.119427,0.207100,0.214016,0.170904,0.157775,0.179299,0.060083,0.113275
공휴일,-5.410588e-02,-0.043601,1.000000,0.815902,-0.252612,0.071894,0.233564,-0.137410,-0.121509,-0.115404,...,-3.300194e-02,-0.205480,-0.130192,-0.119345,-0.198347,-0.105068,-0.085612,-0.138219,-0.041536,-0.091489
공휴일_여부,-4.093609e-02,-0.105724,0.815902,1.000000,-0.338620,0.116822,0.334722,-0.182571,-0.156728,-0.145828,...,-5.719356e-02,-0.250908,-0.160367,-0.139056,-0.227590,-0.135150,-0.096241,-0.165323,-0.052563,-0.107885
최고기온(°C),3.580905e-01,0.364914,-0.252612,-0.338620,1.000000,-0.264699,-0.800711,0.584855,0.544805,0.527746,...,7.004673e-02,0.577905,0.370107,0.441038,0.553821,0.352714,0.422662,0.527605,0.326976,0.376896
patient,-9.904741e-02,-0.005352,0.071894,0.116822,-0.264699,1.000000,0.342664,-0.126705,-0.109379,-0.100625,...,4.013501e-02,-0.182452,-0.108561,-0.110241,-0.173076,-0.082424,-0.094771,-0.161832,-0.082514,-0.096865
년도,-3.874877e-01,-0.073210,0.233564,0.334722,-0.800711,0.342664,1.000000,-0.670146,-0.601534,-0.573723,...,-3.781189e-06,-0.326736,-0.173041,-0.169292,-0.319759,-0.106208,-0.155837,-0.322672,-0.118543,-0.138395
분기,3.049082e-01,-0.075384,-0.137410,-0.182571,0.584855,-0.126705,-0.670146,1.000000,0.980583,0.974292,...,2.782963e-01,-0.059620,-0.239167,-0.142905,0.124075,-0.180362,-0.066236,0.135645,-0.118214,-0.147305
월,3.593554e-01,-0.098426,-0.121509,-0.156728,0.544805,-0.109379,-0.601534,0.980583,1.000000,0.997002,...,2.722574e-01,-0.076997,-0.236516,-0.128518,0.120513,-0.169433,-0.037301,0.130099,-0.102458,-0.122734
주,3.361294e-01,-0.092834,-0.115404,-0.145828,0.527746,-0.100625,-0.573723,0.974292,0.997002,1.000000,...,2.808893e-01,-0.080409,-0.240488,-0.131238,0.115688,-0.176083,-0.042585,0.132055,-0.101119,-0.126917


In [ ]:
# 일단은 이게 최종적으로 만든 파일임!
train_new.to_pickle('./pkl_data/train_외부변수포함_final(0901).pkl')